In [ ]:
import sys

# Load diabetes dataset from sklearn
from sklearn.datasets import load_iris

import datagnosis.logger as log
log.add(sink=sys.stderr, level="INFO")

In [ ]:
X, y = load_iris(return_X_y=True, as_frame=True)
df = X.copy(deep=True)
df['target'] = y
display(df)

In [ ]:

from datagnosis.plugins.core.datahandler import DataHandler
from datagnosis.plugins.core.models.simple_mlp import SimpleMLP
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn

std_scaler = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
X_train = std_scaler.fit_transform(X_train)
X_test = std_scaler.transform(X_test)

datahander = DataHandler(X_train, y_train, batch_size=32)


model = SimpleMLP()
# creating our optimizer and loss function object
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [ ]:
# datagnosis absolute
from datagnosis.plugins import Plugins

plugins = Plugins().list()
print(plugins)


In [ ]:
hcm = Plugins().get(
    "data_iq",
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    lr=learning_rate,
    epochs=10,
    num_classes=3,
    logging_interval=1,
)
hcm.fit(
    datahandler=datahander,
    use_caches_if_exist=True,
)

In [ ]:
hcm_scores = hcm.scores
# print(hcm_scores)
print(hcm.score_names)
hcm.plot_scores(axis=1, plot_type="scatter")

In [ ]:
import pandas as pd
print(hcm.name())
print(hcm.hard_direction())
hardest_5 = hcm.extract_datapoints(method="index", indices=[0,1,2,3,4,5])

display(pd.DataFrame(
    data={
        "indices":hardest_5[0][2],
        f"{X.columns[0]}": hardest_5[0][0].transpose(0,1)[0],
        f"{X.columns[1]}": hardest_5[0][0].transpose(0,1)[1],
        f"{X.columns[2]}": hardest_5[0][0].transpose(0,1)[2],
        f"{X.columns[3]}": hardest_5[0][0].transpose(0,1)[3],
        "labels": hardest_5[0][1],
        "scores": hardest_5[1],
    }
))